## Дедуплициране на данни с Pandas (подготовка на данните)

Този ноутбук се занимава само с подготвянето на данните за демонстрацията по дедуплициране. За преминаване директно към интересната част последвайте [този](./pandas_deduplication_2.ipynb) линк.


In [2]:
import csv
import datetime
from collections import OrderedDict
from faker import Faker
import pandas as pd

In [3]:
fake = Faker()
Faker.seed(0)

In [4]:
header = ["name", "sex", "birthdate", "username", "address", "SSN", "mail"]
rows_number = 100001
output_filename = "over_a_hundred_thousand.csv"

Умишлено ще зададем опцията `unique` за полетата `name`, `SSN`, и `mail`, за да можем да имаме по-добър контрол върху дублираните записи, които ще добавим по-късно.

In [5]:
%%time
with open(output_filename, "wt") as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=header, delimiter=";")
    writer.writeheader()
    for i in range(rows_number):
        writer.writerow({
            "name": fake.unique.name(),
            "sex": fake.random_element(elements=OrderedDict([("M", 0.45), ("F", 0.40), ("", 0.11), ("d", 0.04), ])),
            "birthdate": fake.date(pattern="%d-%m-%Y", end_datetime=datetime.date(2002, 1, 1)),
            "username": fake.profile(fields=["username"]).get("username"),
            "address": fake.unique.address().replace("\n", " "),
            "SSN": fake.unique.ssn(),
            "mail": fake.unique.email(),
        })

CPU times: user 3min 2s, sys: 111 ms, total: 3min 2s
Wall time: 3min 2s


In [6]:
!head generated_rows.csv -n 5

head: cannot open 'generated_rows.csv' for reading: No such file or directory


### Създаване на дубликати

Стратегията ни ще е да вкараме създадените току що данни в pandas, след което да изберем няколко произолно подбрани масиви от редове, всеки с различна степен на дуплициране: 
 - масив на напълно еднакви редове (перфектни дубликати),
 - масив на еднакви SSN, имена и адреси, но различни адреси на електронна поща, 
 - масив на еднакви SSN и имена, но различни адреси и различни адреси на електронна поща
 - масив на еднакви имена, но различни SSN, адреси и адреси на електронна поща
 


In [7]:
df_dataset = pd.read_csv("over_a_hundred_thousand.csv", delimiter=";")
len(df_dataset)

100001

Параметър за размера на извадката с дуплицирани данни (в %):

Параметри за процент дубликати по различните категории:

In [8]:
duplicate_case_pct = {
    "perfect_duplicate": 0.1,               # all columns match
    "mail_diff": 2,                              # only the mail is different
    "mail_addr_diff": 3,                   # mail and address are different
    "diff_all_but_name": 4,             # all columns but name are different
}


Параметърът `replace=False` не позволява на pandas да избере два пъти един и същ ред по време на самплинга, за да сме сигурни, че всеки от записите от `random_sample` фрейма ще попадне в само една от групите дубликати.

In [9]:
df_duplicates = {}
for (k, v) in duplicate_case_pct.items():
    df_duplicates[k] = df_dataset.sample(frac=v/100, replace=False)

In [10]:
df_duplicates["mail_diff"][:5]

,name,sex,birthdate,username,address,SSN,mail
78918,Julie Hicks,M,21-11-1994,ljohnson,84141 Taylor Extensions Suite 591 New Cynthiab...,028-08-7449,yberry@hotmail.com
67865,Stephen Turner,F,13-08-1984,carl28,Unit 2875 Box 8319 DPO AE 32613,282-37-6877,billy30@gmail.com
49220,Anita Morgan,M,24-04-1984,youngderek,"49324 Silva Stravenue Ericaland, DE 34937",407-10-2322,lloydmichael@cortez.info
68368,Stephen Weber,F,24-06-1970,ronaldyu,"6494 Smith Tunnel Suite 748 West Richard, ND 8...",102-52-1561,xwright@orr.com
4294,Noah Delgado,F,13-10-1989,qmoran,"0413 Kelli Springs Suite 661 Port Jennifer, ND...",776-87-0666,cooperdiana@yahoo.com


In [11]:
df_duplicates["mail_diff"]["mail"] = df_duplicates["mail_diff"]["mail"].apply(lambda x: fake.unique.email())
df_duplicates["mail_diff"][:5]

,name,sex,birthdate,username,address,SSN,mail
78918,Julie Hicks,M,21-11-1994,ljohnson,84141 Taylor Extensions Suite 591 New Cynthiab...,028-08-7449,blake97@sullivan-pitts.com
67865,Stephen Turner,F,13-08-1984,carl28,Unit 2875 Box 8319 DPO AE 32613,282-37-6877,ashley99@gmail.com
49220,Anita Morgan,M,24-04-1984,youngderek,"49324 Silva Stravenue Ericaland, DE 34937",407-10-2322,lwilson@simmons.org
68368,Stephen Weber,F,24-06-1970,ronaldyu,"6494 Smith Tunnel Suite 748 West Richard, ND 8...",102-52-1561,tarmstrong@peterson.info
4294,Noah Delgado,F,13-10-1989,qmoran,"0413 Kelli Springs Suite 661 Port Jennifer, ND...",776-87-0666,newtonstephen@yahoo.com


In [12]:
df_duplicates["mail_addr_diff"][:5]

,name,sex,birthdate,username,address,SSN,mail
50416,Carl Robinson,NaN,05-05-2001,garcianatalie,"4702 Acevedo Roads Davismouth, GA 29515",388-01-7256,xbailey@gmail.com
13291,Curtis Huerta III,M,22-12-1995,bakerjames,2277 Martinez Ports Suite 386 South Timothyche...,169-03-5195,oliviacole@johns.com
40305,Bryan Ramirez,M,13-09-1974,jholloway,"38650 Kathryn Shore Apt. 490 Lake Jay, ME 50846",763-18-9788,proberts@brewer.com
81337,Amy Peterson,M,15-05-1998,ashleyramirez,"33066 Robinson Isle Ibarrafurt, SD 51411",200-09-2254,florestim@lozano.com
18778,Benjamin Nelson,d,05-05-1990,amanda21,"8992 Ruth Row Sawyerhaven, UT 27880",152-93-2120,wrightkendra@jordan.com


In [13]:
df_duplicates["mail_addr_diff"]["mail"] = df_duplicates["mail_addr_diff"]["mail"].apply(lambda x: fake.unique.email())
df_duplicates["mail_addr_diff"]["address"] = df_duplicates["mail_addr_diff"]["address"].apply(lambda x: fake.unique.address().replace("\n", " "))
df_duplicates["mail_addr_diff"][:5]

,name,sex,birthdate,username,address,SSN,mail
50416,Carl Robinson,NaN,05-05-2001,garcianatalie,"15484 Jennifer Meadow Suite 320 Sydneystad, DE...",388-01-7256,laura16@gmail.com
13291,Curtis Huerta III,M,22-12-1995,bakerjames,"93348 Howard Mount Jeremyfurt, OR 56442",169-03-5195,gonzalezmary@mueller.net
40305,Bryan Ramirez,M,13-09-1974,jholloway,"824 Taylor Lake Timothyville, CT 64514",763-18-9788,reneehill@yahoo.com
81337,Amy Peterson,M,15-05-1998,ashleyramirez,"414 Bean Dam South John, GA 71281",200-09-2254,tinacooper@gmail.com
18778,Benjamin Nelson,d,05-05-1990,amanda21,"66145 Connie Dale Suite 136 Lake Hectorbury, N...",152-93-2120,adillon@reynolds.com


In [14]:
df_duplicates["diff_all_but_name"][:5]

,name,sex,birthdate,username,address,SSN,mail
13599,Ashley Hebert,F,01-09-1983,meyersherry,"PSC 4017, Box 5037 APO AA 87718",722-52-3884,qfernandez@townsend-hardy.com
8769,Catherine Sullivan,M,17-04-1971,greed,"4284 Geoffrey Cove Apt. 201 New Brandy, VA 49879",519-02-1472,rogerschristopher@yahoo.com
27575,Jacqueline Ford,M,14-06-1971,hillashley,"6453 Christian Well Apt. 378 Heathberg, VT 30975",606-36-1554,heathersnyder@hotmail.com
85151,Reginald Novak,F,04-08-2000,walkerrobert,"49448 Julie Coves South Carlosville, VT 93253",656-80-4821,alexamartinez@gmail.com
60757,Kevin Atkinson,F,28-04-1973,joshuabaker,"950 Chavez Lodge Suite 308 Thomasberg, MS 89042",651-75-2029,warrenmark@hotmail.com


In [15]:
df_duplicates["diff_all_but_name"]["mail"] = df_duplicates["diff_all_but_name"]["mail"].apply(lambda x: fake.unique.email())
df_duplicates["diff_all_but_name"]["address"] = df_duplicates["diff_all_but_name"]["address"].apply(lambda x: fake.unique.address().replace("\n", " "))
df_duplicates["diff_all_but_name"]["birthdate"] = df_duplicates["diff_all_but_name"]["birthdate"].apply(lambda x:  fake.date(pattern="%d-%m-%Y", end_datetime=datetime.date(2002, 1, 1)))
df_duplicates["diff_all_but_name"]["username"] = df_duplicates["diff_all_but_name"]["username"].apply(lambda x: fake.profile(fields=["username"]).get("username"))
df_duplicates["diff_all_but_name"]["SSN"] = df_duplicates["diff_all_but_name"]["SSN"].apply(lambda x: fake.unique.ssn())
df_duplicates["diff_all_but_name"][:5]

,name,sex,birthdate,username,address,SSN,mail
13599,Ashley Hebert,F,18-11-1983,harold61,"57885 Hull Green Jamesport, MA 38079",066-18-2273,judith87@hotmail.com
8769,Catherine Sullivan,M,05-09-1995,bethanyburke,"9824 Beck Rapids Butlerburgh, ND 63117",208-61-3328,donnaking@acosta-stevens.com
27575,Jacqueline Ford,M,08-09-1995,wtodd,Unit 4646 Box 5392 DPO AE 06633,602-69-9934,wbrown@miller-griffith.com
85151,Reginald Novak,F,03-09-1972,ojackson,"92224 Kathy Plain Apt. 906 Monicabury, MT 17246",633-12-7567,tiffany62@hotmail.com
60757,Kevin Atkinson,F,24-02-1983,aaronwashington,"5499 Mcconnell Cape New Gabrielle, OH 90410",731-76-1296,ariasjulian@gmail.com


In [16]:
df_all_duplicates = pd.concat([f for f in df_duplicates.values()], ignore_index=True)
len(df_all_duplicates)

9100

In [17]:
df_final_dataset = pd.concat([df_dataset, df_all_duplicates], ignore_index=True)
len(df_final_dataset)

109101

Методът `DataFrame.sample()`, който използвахме, за да направим случайна извадка може да се използва и за произволно разбъркване на данните в даден фрейм. За целта използваме параметър `frac`:

In [18]:
df_final_dataset = df_final_dataset.sample(frac=1)

In [19]:
df_final_dataset.head(5)

,name,sex,birthdate,username,address,SSN,mail
46035,Sheri Lopez,F,01-05-1979,pgarcia,"191 Bean Parkway Princeborough, NH 15235",381-76-3124,lmorris@jarvis-oneal.com
37168,Andrea Wright,NaN,07-10-1977,mfitzgerald,USNS Morton FPO AA 84762,772-29-1535,samuelpierce@thompson.com
31415,Jon Alvarez,F,13-01-1998,pbartlett,"PSC 7480, Box 6892 APO AP 89421",692-62-1568,marshwhitney@yahoo.com
31548,Joshua Simon,M,08-01-1988,powelljoy,"4145 Carroll Turnpike West Sarahfurt, NE 08571",529-52-4445,rsanchez@thompson-nash.biz
25299,Kelly Chase,F,28-05-1988,georgebowers,"643 Jackson Underpass Apt. 359 Andersontown, N...",692-78-5877,nicholasrusso@gonzalez-byrd.com


In [21]:
df_final_dataset.to_csv("dataset_with_duplicates.csv", sep="|", index=False)